## Finetuning NLI-deberta-v3 model
- This is based on the Prof. Mihai Surdeanu's text book <a href="https://github.com/clulab/gentlenlp/blob/main/notebooks/chap13_classification_bert.ipynb">Gentle NLP Chapter 13 Classification using BERT model</a>
- Modified for NLI evaluation and analysis over SICCK dataset
- Author: Sushma Anand Akoju, Email: sushmaakoju@arizona.edu

In [97]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install 'transformers[torch]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Text Classification Using Transformer Networks (Deberta and Roberta)

Some initialization:

In [2]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [30]:
import os
import pandas as pd
path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/block-wise-data/blocks-dec26/data/15/"
assert os.path.exists(path), "Error"
temp = pd.read_csv(os.path.join(path,"SICCK-zero-shot-analysis-apr24.csv"))
df = temp[temp.columns[1:]]
df.head(), len(df)

(   SICK_id                                  Premise  \
 0       90      a man is jumping into an empty pool   
 1       90  every man is jumping into an empty pool   
 2       90      a man is jumping into an empty pool   
 3       90  every man is jumping into an empty pool   
 4       90   some man is jumping into an empty pool   
 
                               Hypothesis Modifier Premise/Hypothesis/Both  \
 0      a man is jumping into a full pool     NONE                    NONE   
 1      a man is jumping into a full pool    every                 Premise   
 2  every man is jumping into a full pool    every              Hypothesis   
 3  every man is jumping into a full pool    every                    Both   
 4      a man is jumping into a full pool     some                 Premise   
 
   Part of Premise/Hypothesis Modified Ground Truth           GT Modifier Type  \
 0                                NONE  Alternation  Alternation  No Modifiers   
 1                          

In [31]:
df['CompressedGT'].unique()

array(['Contradiction', 'Neutral', 'FE', 'RE'], dtype=object)

In [32]:
label2id = {
    'Contradiction': 0,
    'Neutral': 1, 
    'FE': 2, 
    'RE': 2, 
}
label2id_roberta = {
    'Contradiction': 2,
    'Neutral': 1, 
    'FE': 0, 
    'RE': 0, 
}
df['label'] = df['CompressedGT'].map(lambda x: label2id[x])
df['label4roberta'] = df['CompressedGT'].map(lambda x: label2id_roberta[x])

In [33]:
df['label'].unique(), df['CompressedGT'].unique(), df['label4roberta'].unique()

(array([0, 1, 2]),
 array(['Contradiction', 'Neutral', 'FE', 'RE'], dtype=object),
 array([2, 1, 0]))

Read the train/dev/test datasets and create a HuggingFace `Dataset` object:

## Rolling window (circular array style) splits for 5-fold Cross validation
- to account for the few extra indices that might run beyond the length of data for test or train sets

In [73]:
import math
d = list(range(len(df)))
n = len(d)
test_len = math.ceil(1304/5)
train_len = n-(2*test_len)
val_len = n-train_len - test_len
print(train_len, test_len, val_len)
math.ceil(1304/5), n-(2*261),n, n-test_len

782 261 261


(261, 782, 1304, 1043)

In [52]:
len(d)

1304

In [71]:
def circular_array(starting_index, ending_index, d):
  idx = d
  idx = np.roll(idx, -starting_index)[:(len(idx)-starting_index+ending_index)%len(idx)]

  return idx

In [76]:
len(circular_array(261+test_len,261+test_len+1043, d))

1043

In [78]:
counter = 1
folds = []
columns = ['Premise', 'Hypothesis', 'label','label4roberta', 'CompressedGT', 'Modifier Type', 
           'Modifier',	'Premise/Hypothesis/Both',	'Part of Premise/Hypothesis Modified']
# test_len = 261
for i in range(0,n, test_len):
  # print(i, len(d[i:i+test_len]), len(circular_array(i, i+test_len, d)))
  test = df.iloc[circular_array(i, i+test_len, d)][columns]
  train = df.iloc[circular_array(i+test_len, i+test_len+1043, d)][columns]
  print(len(test), len(train))
  # val = df.iloc[circular_array(i+test_len+1130, i+test_len+1130+test_len, d)][columns]
  counter += 1
  folds.append({"train":train, "test":test})

261 1043
261 1043
261 1043
261 1043
261 1043


In [80]:
len(folds[0]["train"]), len(folds[0]["test"])

(1043, 261)

### Save all data into one excel sheet

In [81]:
output_path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/june12"
for i,fold in enumerate(folds):
  with pd.ExcelWriter(os.path.join(output_path, "fold"+str(i)+".xlsx")) as writer:
    fold["train"].to_excel(writer, sheet_name="train", index=False )
    fold["test"].to_excel(writer, sheet_name="test", index=False )

In [82]:
output_path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/june12"
with pd.ExcelWriter(os.path.join(output_path, "five_folds.xlsx")) as writer:

  for i,fold in enumerate(folds):
      # fold["train"].to_excel(writer, sheet_name="train", index=False )
    fold["test"].to_excel(writer, sheet_name="fold"+str(i), index=False )

### Modifier type distribution each of the test set

In [83]:
with pd.ExcelWriter(os.path.join(output_path, "fold_distribution.xlsx")) as writer:
  for i,fold in enumerate(folds):
    print(i,fold["test"].groupby(["Modifier Type"]).count().reset_index()[["Modifier Type","CompressedGT"]])
    fold["test"].groupby(["Modifier Type"]).count().reset_index()[["Modifier Type","CompressedGT"]].to_excel(writer, sheet_name="fold"+str(i), index=False)

0         Modifier Type  CompressedGT
0  Adjectives/Adverbs           123
1         Existential            57
2            Negation            33
3        No Modifiers             3
4           Universal            45
1         Modifier Type  CompressedGT
0  Adjectives/Adverbs           122
1         Existential            63
2            Negation            29
3        No Modifiers             3
4           Universal            44
2         Modifier Type  CompressedGT
0  Adjectives/Adverbs           117
1         Existential            60
2            Negation            36
3        No Modifiers             3
4           Universal            45
3         Modifier Type  CompressedGT
0  Adjectives/Adverbs           125
1         Existential            56
2            Negation            36
3        No Modifiers             3
4           Universal            41
4         Modifier Type  CompressedGT
0  Adjectives/Adverbs           115
1         Existential            67
2            Negat

### Label-wise distribution in test splits

In [84]:
with pd.ExcelWriter(os.path.join(output_path, "fold_label_distribution.xlsx")) as writer:
  for i,fold in enumerate(folds):
    print(i,fold["test"].groupby(["CompressedGT"]).count().reset_index()[["CompressedGT", "Modifier Type"]])
    fold["test"].groupby(["CompressedGT"]).count().reset_index()[["CompressedGT","Modifier Type"]].to_excel(writer, sheet_name="fold"+str(i), index=False)

0     CompressedGT  Modifier Type
0  Contradiction             76
1        Neutral            185
1     CompressedGT  Modifier Type
0  Contradiction            140
1             FE             60
2        Neutral             50
3             RE             11
2     CompressedGT  Modifier Type
0  Contradiction             90
1             FE             87
2        Neutral             74
3             RE             10
3     CompressedGT  Modifier Type
0  Contradiction            108
1             FE             68
2        Neutral             79
3             RE              6
4     CompressedGT  Modifier Type
0  Contradiction              9
1             FE             44
2        Neutral            205
3             RE              3


### Label-Modifier type count distribution for analysis : to verify the gap or imbalanced distribution

In [85]:
with pd.ExcelWriter(os.path.join(output_path, "fold_label_qtype_distribution.xlsx")) as writer:
  for i,fold in enumerate(folds):
    print(i,fold["test"].groupby(["CompressedGT", "Modifier Type"]).count().reset_index()[["CompressedGT", "Modifier Type", "Modifier"]])
    fold["test"].groupby(["CompressedGT", "Modifier Type"]).count().reset_index()[["CompressedGT","Modifier Type", "Modifier"]].to_excel(writer, sheet_name="fold"+str(i), index=False)

0     CompressedGT       Modifier Type  Modifier
0  Contradiction  Adjectives/Adverbs        42
1  Contradiction         Existential        15
2  Contradiction        No Modifiers         1
3  Contradiction           Universal        18
4        Neutral  Adjectives/Adverbs        81
5        Neutral         Existential        42
6        Neutral            Negation        33
7        Neutral        No Modifiers         2
8        Neutral           Universal        27
1      CompressedGT       Modifier Type  Modifier
0   Contradiction  Adjectives/Adverbs        77
1   Contradiction         Existential        27
2   Contradiction            Negation         8
3   Contradiction        No Modifiers         2
4   Contradiction           Universal        26
5              FE  Adjectives/Adverbs        28
6              FE         Existential        17
7              FE            Negation         5
8              FE        No Modifiers         1
9              FE           Universal         

In [86]:
len(folds)

5

In [87]:
for fold in folds:
  print(len(fold["train"]), len(fold["test"]))

1043 261
1043 261
1043 261
1043 261
1043 261


### Create dtaa splits with premise, hypothesis as well as hypothesis, premise for **Test** set predictions to label:
- Forward Entailment
- Reverse Entailment
- Neutral 

In [88]:
def read_data(data):
    # concatenate title and description, and remove backslashes
    data['text'] = data['Premise'] + " [SEP] " + data['Hypothesis']
    data['text'] = data['text'].str.replace('\\', ' ', regex=False)
    return data

In [89]:
def read_data_reverse(data):
    # concatenate title and description, and remove backslashes
    data['text'] = data['Hypothesis'] + " [SEP] " + data['Premise']
    data['text'] = data['text'].str.replace('\\', ' ', regex=False)
    return data

### Compute metrics for validation and test

In [90]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred), 'recall': recall_score(y_true, y_pred, average='micro'), 
            'f1':f1_score(y_true, y_pred, average='micro'), 'precision':precision_score(y_true, y_pred, average='micro')}
def compute_test_metrics(y_true, y_pred):
    return {'accuracy': accuracy_score(y_true, y_pred), 'recall': recall_score(y_true, y_pred, average='micro'), 
            'f1':f1_score(y_true, y_pred, average='micro'), 'precision':precision_score(y_true, y_pred, average='micro')}

### To include FE, RE and Neutral label calculation and scores for **Test**

In [91]:
from sklearn.metrics import classification_report
def test_eval(trainer, ds, fold, model_name ):
  test_ds = ds['test'].map(
      tokenize,
      batched=True,
      remove_columns=['Premise', 'Hypothesis', 'text'],
  )
  rev_test_ds = ds['rev_test'].map(
      tokenize,
      batched=True,
      remove_columns=['Premise', 'Hypothesis', 'text'],
  )
  test_ds.to_pandas()
  output = trainer.predict(test_ds)
  rev_scores = trainer.predict(rev_test_ds)

  y_true = output.label_ids
  y_preds = np.argmax(output.predictions, axis=-1)
  y_rev_score_preds = np.argmax(rev_scores.predictions, axis=-1)
  labels = []

  if "roberta" in model_name:
    for i in range(len(y_preds)):
          if y_preds[i] == 0:
            labels.append("FE")
          elif y_preds[i] == 2:
            labels.append("Negation")
          else:
            if y_rev_score_preds[i] == 1:
              labels.append("RE")
            else:
              labels.append("Neutral")
    print(classification_report(y_true, y_preds, labels=[0, 1, 2]))
    
  else:
    for i in range(len(y_preds)):
      if y_preds[i] == 1:
        labels.append("FE")
      elif y_preds[i] == 0:
        labels.append("Negation")
      else:
        if y_rev_score_preds[i] == 1:
          labels.append("RE")
        else:
          labels.append("Neutral")
    print(classification_report(y_true, y_preds, labels=[0, 1, 2]))
  res = compute_test_metrics(y_true, y_preds)
  res['fold'] = fold
  res['model_name'] = model_name
  return y_true, y_preds, res, labels

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-base', num_labels=3)
# tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-base')

### Get this_train, this_validation & this_test set from a this_fold

In [92]:
def get_dataset(fold, model_name):
  labels_header_name = 'label'
  if "roberta" in model_name:
    labels_header_name = 'label4roberta'
    
  columns = ['Premise', 'Hypothesis', labels_header_name]

  train_df = read_data(fold["train"][columns])
  test_df = read_data(fold["test"][columns])
  rev_test_df = read_data_reverse(fold["test"][columns])
  print(test_df.columns)

  train_df, eval_df = train_test_split(train_df, train_size=0.9)
  train_df.reset_index(inplace=True, drop=True)
  eval_df.reset_index(inplace=True, drop=True)
  test_df.reset_index(inplace=True, drop=True)
  rev_test_df.reset_index(inplace=True, drop=True)

  print(f'train rows: {len(train_df.index):,}')
  print(f'eval rows: {len(eval_df.index):,}')
  print(f'test rows: {len(test_df.index):,}')
  print(f'test rows: {len(rev_test_df.index):,}')

  ds = DatasetDict()
  ds['train'] = Dataset.from_pandas(train_df)
  ds['validation'] = Dataset.from_pandas(eval_df)
  ds['test'] = Dataset.from_pandas(test_df)
  ds['rev_test'] = Dataset.from_pandas(rev_test_df)

  print(ds)
  return ds, test_df, rev_test_df

### CustomTrainer for CrossEntropyLoss but we train for both custom and default Trainer classes in HuggingFace
- Note: we did not see any difference between the two

In [93]:

import torch
from torch import nn
from transformers import Trainer
from accelerate import Accelerator

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss_fct.to('cuda')
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

## Fine tune "cross-encoder/nli-deberta-v3-base" for 1304 examples for five folds.
- Use rolling window for train-test folds
- split validation from training (same size as test)
- test size: 261

### Tokenize & Train one model at a time for all folds

In [94]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model_name = ""
model_names =["cross-encoder/nli-deberta-v3-base",	"ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"]
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(examples):
    return tokenizer(examples['text'], truncation=True)

def train(model_name, this_path, folds):
  epochs = [4, 8]
  batch_sizes = [8,16,32]
  m = model_name.split("/")[1]
  all_scores = []
  # tokenizer = AutoTokenizer.from_pretrained(model_name)
  # model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
  # tokenizer = AutoTokenizer.from_pretrained(model_name)
  for num_epochs in epochs:
    for batch_size in batch_sizes:

      for i,fold in enumerate(folds):
          print("\n***********************************************************************************\n")
          print("\n**************** The number of epochs, batch_size and fold respectively are: ",num_epochs, batch_size, i,"************************\n")
          model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

          ds, test_df, rev_test_df = get_dataset(fold,model_name)
          train_ds = ds['train'].map(
            tokenize, batched=True,
            remove_columns=['Premise', 'Hypothesis', 'text'],
          )
          eval_ds = ds['validation'].map(
              tokenize,
              batched=True,
              remove_columns=['Premise', 'Hypothesis', 'text'],
          )

          weight_decay = 0.01
          tx_model_name = f'{model_name}-sequence-classification'

          training_args = TrainingArguments(
              output_dir=os.path.join(output_path,m+"_"+str(num_epochs)+str(batch_size)+"trainer"),
              log_level='error',
              num_train_epochs=num_epochs,
              per_device_train_batch_size=batch_size,
              per_device_eval_batch_size=batch_size,
              evaluation_strategy='epoch',
              weight_decay=weight_decay,
          )
          trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            compute_metrics=compute_metrics,
            tokenizer=tokenizer,
          )
          trainer.train()

          customTrainer  = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            compute_metrics=compute_metrics,
            tokenizer=tokenizer,
          )

          customTrainer.train()
          y_true, y_pred, results, labels = test_eval(trainer, ds, i, model_name )
          y_true1, y_pred1, results1, labels1 = test_eval(customTrainer, ds, i, model_name )

          all_scores.append(results)
          fold["test"]["label"]= y_true
          fold["test"]["predictions"] = y_pred
          fold["test"]["predictions2"] = y_pred1
          fold["test"]["text"] = test_df['text']
          fold["test"]["pred_labels"] =  labels
          filename = "five_"+m+"_"+str(num_epochs)+"_"+str(batch_size)+"_"+str(i)+"_test.xlsx"
          fold["test"].to_csv(os.path.join(this_path, filename))
  return all_scores

In [95]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [51]:
# if tokenizer:
#   del tokenizer
# if model:
#   del model

### "cross-encoder/nli-deberta-v3-base"

In [96]:
from accelerate import Accelerator
all_scores = []
predictions = []
# if tokenizer:
#   del tokenizer
# if model:
#   del model
#for model_name in model_names:
model_name = model_names[0]
m = model_name.split("/")[1]
this_path = os.path.join(output_path, m)
if not os.path.exists(this_path):
  os.mkdir(this_path)
assert os.path.exists(this_path), "%s Path does not exists!"%(this_path)

# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)
all_scores.append(train(model_name, this_path, folds))

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



***********************************************************************************


**************** The number of epochs, batch_size and fold respectively are:  4 8 0 ************************

Index(['Premise', 'Hypothesis', 'label', 'text'], dtype='object')
train rows: 938
eval rows: 105
test rows: 261
test rows: 261
DatasetDict({
    train: Dataset({
        features: ['Premise', 'Hypothesis', 'label', 'text'],
        num_rows: 938
    })
    validation: Dataset({
        features: ['Premise', 'Hypothesis', 'label', 'text'],
        num_rows: 105
    })
    test: Dataset({
        features: ['Premise', 'Hypothesis', 'label', 'text'],
        num_rows: 261
    })
    rev_test: Dataset({
        features: ['Premise', 'Hypothesis', 'label', 'text'],
        num_rows: 261
    })
})


<ipython-input-88-3e308bfb0585>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['Premise'] + " [SEP] " + data['Hypothesis']
<ipython-input-88-3e308bfb0585>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].str.replace('\\', ' ', regex=False)
<ipython-input-88-3e308bfb0585>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Map:   0%|          | 0/938 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       0.22      0.89      0.36         9
           1       0.83      0.78      0.80       205
           2       0.39      0.28      0.33        47

    accuracy                           0.69       261
   macro avg       0.48      0.65      0.49       261
weighted avg       0.73      0.69      0.70       261



Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       0.22      0.89      0.36         9
           1       0.83      0.78      0.80       205
           2       0.39      0.28      0.33        47

    accuracy                           0.69       261
   macro avg       0.48      0.65      0.49       261
weighted avg       0.73      0.69      0.70       261



In [98]:
all_scores_deberta = all_scores

In [99]:
torch.cuda.empty_cache() 

In [100]:
all_scores

[[{'accuracy': 0.8697318007662835,
   'recall': 0.8697318007662835,
   'f1': 0.8697318007662835,
   'precision': 0.8697318007662835,
   'fold': 0,
   'model_name': 'cross-encoder/nli-deberta-v3-base'},
  {'accuracy': 0.7126436781609196,
   'recall': 0.7126436781609196,
   'f1': 0.7126436781609196,
   'precision': 0.7126436781609196,
   'fold': 1,
   'model_name': 'cross-encoder/nli-deberta-v3-base'},
  {'accuracy': 0.7816091954022989,
   'recall': 0.7816091954022989,
   'f1': 0.781609195402299,
   'precision': 0.7816091954022989,
   'fold': 2,
   'model_name': 'cross-encoder/nli-deberta-v3-base'},
  {'accuracy': 0.7777777777777778,
   'recall': 0.7777777777777778,
   'f1': 0.7777777777777778,
   'precision': 0.7777777777777778,
   'fold': 3,
   'model_name': 'cross-encoder/nli-deberta-v3-base'},
  {'accuracy': 0.6628352490421456,
   'recall': 0.6628352490421456,
   'f1': 0.6628352490421456,
   'precision': 0.6628352490421456,
   'fold': 4,
   'model_name': 'cross-encoder/nli-deberta-v3

### "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"

Tokenize the texts:

In [ ]:
# from transformers.models.deberta.modeling_deberta import DebertaModel, DebertaPreTrainedModel

In [ ]:
# from transformers import DebertaForSequenceClassification
# model = DebertaForSequenceClassification.from_pretrained(models[0])
# model.train()

Create the transformer model:

In [ ]:
# from torch import nn
# from transformers.modeling_outputs import SequenceClassifierOutput

# from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# #BERT, SentenceTransformer pretrained model
# # https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

# class BertForSequenceClassification(BertPreTrainedModel):
#     def __init__(self, config):
#         super().__init__(config)
#         self.num_labels = config.num_labels
#         self.bert = BertModel(config)
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.classifier = nn.Linear(config.hidden_size, config.num_labels)
#         self.init_weights()
        
#     def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
#         outputs = self.bert(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             **kwargs,
#         )
#         cls_outputs = outputs.last_hidden_state[:, 0, :]
#         cls_outputs = self.dropout(cls_outputs)
#         logits = self.classifier(cls_outputs)
#         loss = None
#         if labels is not None:
#             loss_fn = nn.CrossEntropyLoss()
#             loss = loss_fn(logits, labels)
#         return SequenceClassifierOutput(
#             loss=loss,
#             logits=logits,
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#         )

In [ ]:
# sliding window folds 
# 15 
# 10% 
# 130 val
# 130 test
# 1000 train

# 1304 indices
# window size 130

# for fold in folds:
#   train, test, val = fold


In [ ]:
# from transformers import AutoConfig

# config = AutoConfig.from_pretrained(
#     transformer_name,
#     num_labels=len(labels),
# )

# model = (
#     BertForSequenceClassification
#     .from_pretrained(transformer_name, config=config)
# )

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Create the trainer object and train:

In [ ]:
# from sklearn.metrics import accuracy_score

# def compute_metrics(eval_pred):
#     y_true = eval_pred.label_ids
#     y_pred = np.argmax(eval_pred.predictions, axis=-1)
#     return {'accuracy': accuracy_score(y_true, y_pred)}